In [ ]:
# 필요 라이브러리부터 설치할께요.
!pip install konlpy pandas seaborn gensim wordcloud

In [ ]:
## import sklearn
import numpy as np
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
from sklearn.linear_model import SGDClassifier
from konlpy.tag import Mecab
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
mecab = Mecab()
#fm.findSystemFonts()
plt.rcParams['font.family']= ["NanumGothicCoding"]
plt.rcParams["axes.unicode_minus"]=False
# GPU 환경 설정하기
# assert tf.test.is_gpu_available() == True, 'GPU 설정을 확인하세요.'
print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_logical_devices('GPU'))

# 1. 데이터 가져오기

In [ ]:
spam = pd.read_csv('spam_1.csv')
# train validation set으로 분리합니다.
x_data = spam['text']
y_data = spam['label']
print('메일 본문의 개수: {}'.format(len(x_data)))
print('레이블의 개수: {}'.format(len(y_data)))


In [ ]:
spam_test = pd.read_csv('spam_test_text.csv')
# train validation set으로 분리합니다.
x_t = spam_test['text']
print('메일 본문의 개수: {}'.format(len(x_t)))


In [ ]:
# Sequence Vectorize 1
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

TOP_K = 20000
MAX_SEQUENCE_LENGTH = 500

def sequence_vectorize(test_texts):
    # Create vocabulary with training texts.
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(test_texts)

    # Vectorize training and validation texts.
    x_train = tokenizer.texts_to_sequences(test_texts)

    # Get max sequence length.
    max_length = len(max(x_train, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH

    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated
    # at the beginning.
    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    return x_train, tokenizer.word_index

x_t2, word_to_index = sequence_vectorize(x_t)
print(x_t2)
#print(word_to_index)

print("훈련 데이터의 크기(shape):", x_t2.shape)

In [ ]:
# Sequence Vectorize 2 

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_t)
x_train_encodedt = tokenizer.texts_to_sequences(x_t)
print(x_train_encodedt[:5])

word_to_indext = tokenizer.word_index
#print(word_to_index)

threshold = 2
total_cnt = len(word_to_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq+value
    if value<threshold:
        rare_cnt=rare_cnt+1
        rare_freq=rare_freq+value
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합(vocabulary)에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

vocab_size = len(word_to_indext)+1
print('단어 집합의 크기: {}'.format((vocab_size)))

print('메일의 최대 길이 : %d' % max(len(sample) for sample in x_train_encodedt))
print('메일의 평균 길이 : %f' % (sum(map(len, x_train_encodedt))/len(x_train_encodedt)))
plt.hist([len(sample) for sample in x_t], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

max_len = 23
x_t_padded = pad_sequences(x_train_encodedt, maxlen = max_len)
print("훈련 데이터의 크기(shape):", x_t_padded.shape)


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=0, stratify=y_data)

print('--------훈련 데이터의 비율-----------')
print(f'정상 메일 = {round(y_train.value_counts()[0]/len(y_train) * 100,3)}%')
print(f'스팸 메일 = {round(y_train.value_counts()[1]/len(y_train) * 100,3)}%')

In [ ]:
# Sequence Vectorize 1
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

TOP_K = 20000
MAX_SEQUENCE_LENGTH = 500

def sequence_vectorize(train_texts, val_texts):
    # Create vocabulary with training texts.
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts)

    # Vectorize training and validation texts.
    x_train = tokenizer.texts_to_sequences(train_texts)
    x_val = tokenizer.texts_to_sequences(val_texts)

    # Get max sequence length.
    max_length = len(max(x_train, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH

    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated
    # at the beginning.
    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    x_val = sequence.pad_sequences(x_val, maxlen=max_length)
    return x_train, x_val, tokenizer.word_index

x_train2, x_val2, word_to_index = sequence_vectorize(x_train, x_val)
print(x_train2)
print(x_val2)
#print(word_to_index)

print("훈련 데이터의 크기(shape):", x_train2.shape)
print("훈련 데이터의 크기(shape):", x_val2.shape)

In [ ]:
# Sequence Vectorize 2 

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
x_train_encoded = tokenizer.texts_to_sequences(x_train)
print(x_train_encoded[:5])

word_to_index = tokenizer.word_index
#print(word_to_index)

threshold = 2
total_cnt = len(word_to_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq+value
    if value<threshold:
        rare_cnt=rare_cnt+1
        rare_freq=rare_freq+value
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합(vocabulary)에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

vocab_size = len(word_to_index)+1
print('단어 집합의 크기: {}'.format((vocab_size)))

print('메일의 최대 길이 : %d' % max(len(sample) for sample in x_train_encoded))
print('메일의 평균 길이 : %f' % (sum(map(len, x_train_encoded))/len(x_train_encoded)))
plt.hist([len(sample) for sample in x_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

max_len = 23
x_train_padded = pad_sequences(x_train_encoded, maxlen = max_len)
print("훈련 데이터의 크기(shape):", x_train_padded.shape)


In [ ]:
from nltk.stem import PorterStemmer
def preprocess(x):
    porter=PorterStemmer()
    word_map={}
    cnt=0
    for i in range(len(x.values)):
        tempArr=x.values[i].split(' ')
        for j in range(len(tempArr)):
            tempArr[j]=porter.stem(tempArr[j])
            if tempArr[j] in word_map:
                continue
            else:
                word_map[tempArr[j]]=cnt
                cnt=cnt+1
        x.values[i]=np.array(tempArr)
    return x, word_map
x_data2, word_map = preprocess(x_data)
print("x_data2", x_data2.shape)

In [ ]:
def convertToFeatureVector(x):
    x_len=len(x.values)
    word_map_len=len(word_map.keys())
    retArr=[]
    for i in range(len(x.values)):
        temp=np.zeros(word_map_len)
        for j in range(len(x.values[i])):
            idx = word_map.get(x.values[i][j])
            temp[idx]=1
        retArr.append(temp)
    return np.array(retArr)
x_feature_vec=convertToFeatureVector(x_data2)
print(x_feature_vec[10])

In [ ]:
x_train3, x_test3, y_train3, y_test3 = train_test_split(x_feature_vec, y_data, stratify=y_data, test_size=0.2, random_state=2022)

In [ ]:
x_train3[0]

# 2. Machine Learning(N-grams)
## 2-1. Naive Bayes

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(x_train3, y_train3)
print("score : ", round(clf.score(x_train3, y_train3)*100,2))

In [ ]:
#print("test score : ", round(clf.score(x_test3, y_test3)*100,2))
prediction3 = clf.predict(x_t_padded)

# 3. Deep Learning(Sequence)
## 3-1. Conv1D
## 3-2. LSTM
## 3-3. RNN

In [ ]:
'''
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential
embedding_dim = 32
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train3, y_train3, epochs=4, batch_size=64, validation_split=0.2)

x_test_encoded3 = tokenizer.texts_to_sequences(x_val3)
x_test_padded3 = pad_sequences(x_test_encoded3, maxlen = max_len)
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test_padded3, y_val3)[1]))
'''

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential

embedding_dim = 32
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.fit(x_train2, y_train, epochs=4, batch_size=64, validation_split=0.2)

In [ ]:
prediction4 = model.predict(x_t_padded)
prediction = model.predict(x_t)


In [ ]:
x_test_encoded2 = tokenizer.texts_to_sequences(x_val)
x_test_padded2 = pad_sequences(x_test_encoded2, maxlen = max_len)
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test_padded2, y_val)[1]))

In [ ]:
submission2 = pd.read_csv('spam_test_text.csv')
submission2['label']=y_predict
submission2['label'].unique()
submission2.head()
submission2['label'].loc[submission2['label']=='ham']
submission2['label'].loc[submission2['label']<0.1]
submission2['label'].loc[submission2['label']==0]='ham'
submission2['label'].loc[submission2['label']!='ham']='spam'
submission2.to_csv('/aihub/data/sub9.csv', index=False)


----submission

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

embedding_dim = 32
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=30,
                   verbose=1,
                   restore_best_weights=True)
model.fit(x_train_padded, y_train, epochs=4, batch_size=64, validation_split=0.2, callbacks=[es])

In [ ]:
pred2 = model.predict(X_train_padded2)
x_test_encoded = tokenizer.texts_to_sequences(x_val)
x_test_padded = pad_sequences(x_test_encoded, maxlen = max_len)
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test_padded, y_val)[1]))

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
data = pd.read_csv('spam_1.csv')
print('총 샘플의 수 :',len(data))
X_data = data['text']
Y_data = data['label']
print('메일 본문의 개수: {}'.format(len(x_data)))
print('레이블의 개수: {}'.format(len(y_data)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=0, stratify=y_data)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_encoded = tokenizer.texts_to_sequences(X_train)
print(X_train_encoded[:5])
word_to_index = tokenizer.word_index


In [ ]:
threshold = 2
total_cnt = len(word_to_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합(vocabulary)에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
vocab_size = len(word_to_index) + 1
print('단어 집합의 크기: {}'.format((vocab_size)))

In [ ]:
print('메일의 최대 길이 : %d' % max(len(sample) for sample in X_train_encoded))
print('메일의 평균 길이 : %f' % (sum(map(len, X_train_encoded))/len(X_train_encoded)))
plt.hist([len(sample) for sample in X_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
max_len = 23
X_train_padded = pad_sequences(X_train_encoded, maxlen = max_len)
print("훈련 데이터의 크기(shape):", X_train_padded.shape)

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential

embedding_dim = 32
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train_padded, y_train, epochs=4, batch_size=64, validation_split=0.2)

X_test_encoded = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_encoded, maxlen = max_len)
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test_padded, y_test)[1]))

In [ ]:
data2 = pd.read_csv('spam_test_text.csv')
print('총 샘플의 수 :',len(data2))
X_data2 = data2['text']

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_data2)
X_train_encoded2 = tokenizer.texts_to_sequences(X_data2)
print(X_train_encoded2[:5])
word_to_index2 = tokenizer.word_index
#print(word_to_index)

In [ ]:
threshold = 2
total_cnt = len(word_to_index2) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합(vocabulary)에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
vocab_size2 = len(word_to_index2) + 1
print('단어 집합의 크기: {}'.format((vocab_size2)))

In [ ]:
print('메일의 최대 길이 : %d' % max(len(sample) for sample in X_train_encoded2))
print('메일의 평균 길이 : %f' % (sum(map(len, X_train_encoded2))/len(X_train_encoded2)))
plt.hist([len(sample) for sample in X_data2], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
max_len = 22
X_train_padded2 = pad_sequences(X_train_encoded2, maxlen = max_len)
print("훈련 데이터의 크기(shape):", X_train_padded2.shape)

In [ ]:
import nltk
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
txt=' '.join(spam.text.explode())
nltk_spam = nltk.Text(' '.join(spam.text.explode()))
print(nltk_spam)

In [ ]:
# 태깅 클래스를 활용하여 형태소/명사를 추출합니다.
morphs_spam = mecab.morphs(txt)
print(len(morphs_spam))
morphs_spam[:10]

In [ ]:
nouns_spam = mecab.nouns(txt)
print(len(nouns_spam))
nouns_spam[:10]

In [ ]:
pos_spam = mecab.pos(txt)
print(len(pos_spam))
print(pos_spam[:3])
print(mecab.pos(txt,flatten=False)[:3])
print(mecab.pos(txt,join=True)[:3])

In [ ]:
from nltk import bigrams

In [ ]:
nltk_morphs_spam = nltk.Text(morphs_spam)
nltk_nouns_spam = nltk.Text(nouns_spam)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv=CountVectorizer()

In [ ]:
morphs_ft=cv.fit_transform(nltk_morphs_spam)
morphs_ft.shape
!pip install beautifulsoup4


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
import pickle
import re
import numpy as np


In [ ]:
korean_stopwords_path = "./korean_stopwords.txt"

# 텍스트 파일을 오픈합니다.
with open(korean_stopwords_path, encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

In [ ]:
def clean_english_documents(documents):
    #텍스트 정제 (HTML 태그 제거)
    for i, document in enumerate(documents):
        document = BeautifulSoup(document, 'html.parser').text 
        documents[i] = document

    #텍스트 정제 (특수기호 제거)
    for i, document in enumerate(documents):
        document = re.sub(r'[^ A-Za-z]', '', document) #특수기호 제거, 정규 표현식 
        documents[i] = document

    #텍스트 정제 (불용어 제거)
    nltk.download('punkt')
    nltk.download('stopwords')
    for i, document in enumerate(documents):
        clean_words = []
        for word in nltk.tokenize.word_tokenize(document):
            word = word.lower()
            if word not in nltk.corpus.stopwords.words('english'): #불용어 제거
                clean_words.append(word)
        document = ' '.join(clean_words)
        documents[i] = document

    #텍스트 정제 (형태소 분석)
    nltk.download('averaged_perceptron_tagger')
    for i, document in enumerate(documents):
        clean_words = []
        for word in nltk.tag.pos_tag(nltk.tokenize.word_tokenize(document)):
            if word[1] in ['NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'JJ', 'JJR', 'JJS']: #명사, 동사, 형용사
                clean_words.append(word[0])
        document = ' '.join(clean_words)
        document = document.lower()
        documents[i] = document

    #텍스트 정제 (어간 추출)
    for i, document in enumerate(documents):
        clean_words = []
        for word in nltk.tokenize.word_tokenize(document):
            word = word.lower()
            stemmer = nltk.stem.snowball.SnowballStemmer('english')
            word = stemmer.stem(word) #어간 추출
            clean_words.append(word)
        document = ' '.join(clean_words)
        documents[i] = document

    return documents

In [ ]:
spam = pd.read_csv('spam_1.csv')
# train validation set으로 분리합니다.
x_data = spam['text']
y_data = spam['label']
print('메일 본문의 개수: {}'.format(len(x_data)))
print('레이블의 개수: {}'.format(len(y_data)))


In [ ]:
import re
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
#from pykospacing import Spacing
from collections import Counter